<a href="https://colab.research.google.com/github/kmayutrisna/SA_IMDB/blob/main/Glove_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary library
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, GRU, Bidirectional, concatenate, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Get the dataset from gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Display top of data
data =pd.read_csv("drive/My Drive/data/IMDB Dataset.csv")

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data['review']

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

In [ ]:
data['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Text preprocessing
def preprocess_text(text):

    # Text cleaning
    text = re.sub('[^a-zA-Z]', ' ', text)

    #Single character removal
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)

    #Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)

    # Lowercasing
    text = text.lower()

    # Lowercasing
    text = text.split()

    # Stopword removal
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if not word in stop_words]

    # Join the tokens back to a single string
    text = ' '.join(text)
    return text

In [ ]:
# Preprocess the text column
data['review'] = data['review'].apply(preprocess_text)

In [ ]:
data['review'][0]

'one reviewers mentioned watching oz episode hooked right exactly happened br br first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use word br br called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home many aryans muslims gangstas latinos christians italians irish scuffles death stares dodgy dealings shady agreements never far away br br would say main appeal show due fact goes shows dare forget pretty pictures painted mainstream audiences forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards sold nickel inmates kill order get away well mannered middle class inmates turned prison bitches due

In [ ]:
data.head()

,review,sentiment
0,one reviewers mentioned watching oz episode ho...,positive
1,wonderful little production br br filming tech...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically family little boy jake thinks zombie...,negative
4,petter mattei love time money visually stunnin...,positive


In [ ]:
# Convert sentiment to binary labels
data['sentiment'] = data['sentiment'].map({'positive': 1, 'negative': 0})
#data = data.sample(frac=1).reset_index(drop=True)  # Shuffling the dataset

In [ ]:
data.head()

,review,sentiment
0,one reviewers mentioned watching oz episode ho...,1
1,wonderful little production br br filming tech...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake thinks zombie...,0
4,petter mattei love time money visually stunnin...,1


In [ ]:
X = data['review'].values
y = data['sentiment'].values

In [ ]:
# Splitting the dataset

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.111, random_state=42)
#hasilnya 40005

In [ ]:
# Print the sizes of each set
print("Training set size:", len(X_train))
print("Validation set size:", len(X_val))
print("Testing set size:", len(X_test))

Training set size: 40005
Validation set size: 4995
Testing set size: 5000


In [ ]:
X_train

array(['one movies get better every time see packed many original unconventional ideas always find new detail sabu subsequent movies see unlucky monkey yet ones great failure chance humanism play great roles cutting montage inventive artistic without movie art picture highly entertaining one comparing run lola run keep mind dangan ranna made years shown german tv early probable served inspiration tom tykwer movie way around complementary reviews add like acting ending much movie lot fun many ways manages deliver message without annoying pretentious',
       'dollars wedding ring scene riot also guffawed scene bar hyde snorted horse radish flipped bird japanese guy said pearl harbor buddy think iq higher sure count high br br funny thing line requirement seems though would bash making comments long short hope make words get line br br still two lines short pardon think drink drink think depends whether hands faster mind good think made line limit thanks reading',
       'went see movie 

In [ ]:
X_train[0]

'one movies get better every time see packed many original unconventional ideas always find new detail sabu subsequent movies see unlucky monkey yet ones great failure chance humanism play great roles cutting montage inventive artistic without movie art picture highly entertaining one comparing run lola run keep mind dangan ranna made years shown german tv early probable served inspiration tom tykwer movie way around complementary reviews add like acting ending much movie lot fun many ways manages deliver message without annoying pretentious'

In [ ]:
X_train[40004]

'three stooges always many actors loved love every one shorts made love six stooges curly shemp moe larry joe curly joe shorts hilarious also star many great actors actresses lot many shorts opinion three stooges greatest actors ever time funniest comedy team br br good three stooges short funny cast includes christine mcintyre symona boniface gino corrado fred kelsey sam flint chester conklin theodore lorch lynton brent judy malcolm vernon dent john tyrrell heinie conklin bess flowers stooges performed well short recommend one'

In [ ]:
y_train[0]

1

In [ ]:
y_train[40004]

1

In [ ]:
# Tokenizing the texts
vocab_size = 10000  # Maximum number of words to keep in the vocabulary
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)

In [ ]:
X_train

array(['one movies get better every time see packed many original unconventional ideas always find new detail sabu subsequent movies see unlucky monkey yet ones great failure chance humanism play great roles cutting montage inventive artistic without movie art picture highly entertaining one comparing run lola run keep mind dangan ranna made years shown german tv early probable served inspiration tom tykwer movie way around complementary reviews add like acting ending much movie lot fun many ways manages deliver message without annoying pretentious',
       'dollars wedding ring scene riot also guffawed scene bar hyde snorted horse radish flipped bird japanese guy said pearl harbor buddy think iq higher sure count high br br funny thing line requirement seems though would bash making comments long short hope make words get line br br still two lines short pardon think drink drink think depends whether hands faster mind good think made line limit thanks reading',
       'went see movie 

In [ ]:
# Converting text sequences to integer sequences
train_seq = tokenizer.texts_to_sequences(X_train)
val_seq = tokenizer.texts_to_sequences(X_val)
test_seq = tokenizer.texts_to_sequences(X_test)

In [ ]:
train_seq[0]

[4,
 25,
 16,
 45,
 73,
 7,
 12,
 2677,
 33,
 103,
 6447,
 870,
 107,
 70,
 72,
 1380,
 3559,
 25,
 12,
 8785,
 2991,
 134,
 532,
 17,
 1861,
 455,
 174,
 17,
 462,
 2194,
 3819,
 3995,
 1453,
 102,
 2,
 359,
 307,
 411,
 305,
 4,
 3979,
 365,
 6894,
 365,
 266,
 206,
 21,
 60,
 468,
 862,
 127,
 279,
 2604,
 2678,
 686,
 2,
 23,
 83,
 688,
 570,
 5,
 36,
 155,
 14,
 2,
 71,
 135,
 33,
 621,
 857,
 1475,
 628,
 102,
 507,
 1901]

In [ ]:
import numpy as np
train_seq = np.array(train_seq)  # Convert train_seq to a numpy array
print(train_seq.shape)  # Print the shape of train_seq

lengths = [len(seq) for seq in train_seq]
print(min(lengths))  # Print the minimum sequence length
print(max(lengths))  # Print the maximum sequence length

(40005,)
3
1086


<ipython-input-13-2790d17abbec>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_seq = np.array(train_seq)  # Convert train_seq to a numpy array


In [ ]:
train_seq

array([list([4, 25, 16, 45, 73, 7, 12, 2677, 33, 103, 6447, 870, 107, 70, 72, 1380, 3559, 25, 12, 8785, 2991, 134, 532, 17, 1861, 455, 174, 17, 462, 2194, 3819, 3995, 1453, 102, 2, 359, 307, 411, 305, 4, 3979, 365, 6894, 365, 266, 206, 21, 60, 468, 862, 127, 279, 2604, 2678, 686, 2, 23, 83, 688, 570, 5, 36, 155, 14, 2, 71, 135, 33, 621, 857, 1475, 628, 102, 507, 1901]),
       list([2047, 1877, 1676, 47, 5069, 19, 47, 1285, 4958, 1470, 3057, 754, 104, 186, 3929, 6556, 1709, 27, 7357, 1844, 142, 1309, 189, 1, 1, 58, 56, 228, 84, 59, 9, 8094, 123, 638, 91, 213, 311, 22, 533, 16, 228, 1, 1, 48, 31, 290, 213, 27, 2767, 2767, 27, 5653, 584, 760, 4237, 206, 6, 27, 21, 228, 5283, 1057, 746]),
       list([302, 12, 2, 215, 12, 3382, 537, 622, 92, 3487, 534, 707, 367, 1, 1, 7986, 61, 43, 1120, 40, 307, 40, 334, 665, 300, 11, 11, 15, 2, 445, 326, 1955, 2768, 1784, 44, 609, 259, 124, 1589, 1074, 12, 266, 262, 1619, 1297, 2, 893, 62, 15, 727, 2, 2768, 2569, 8471, 1166, 1166, 1023, 5362, 335, 699, 

In [ ]:
min_length_index = np.argmin(lengths)
max_length_index = np.argmax(lengths)

sequence_with_min_length = X_train[min_length_index]
sequence_with_max_length = X_train[max_length_index]

print("Sequence with minimum length:", sequence_with_min_length)
print("Sequence with maximum length:", sequence_with_max_length)

Sequence with minimum length: script story mess
Sequence with maximum length: sign lost highway says br br major spoilers ahead br br already knew br br since great deal people apparently get point movie like contribute interpretation plot makes perfect sense others pointed one single viewing movie sufficient dvd md cheat looking david lynch top hints unlocking md upon second third viewing please br br first mulholland drive downright brilliant masterpiece kind movie refuse leave head often comments dvds accurate vogue gets inside head stays really hit mark br br david lynch deserves praise creating movie beautifully stylish look cinematography wise great acting esp naomi watts haunting soundtrack badalamenti dream like quality top also manages involve viewer way movies last time saw movie leave mind everyone felt compelled talk write regardless whether liked hated br br allright enough time justify statements br br people gone effort try piece plot together come conclusion first half 

In [ ]:
length_train_seq_1 = len(train_seq[2])

print(length_train_seq_1)

144


In [ ]:
# Padding sequences
#max_sequence_length = max([len(seq) for seq in train_seq])
max_length = 100  # Maximum sequence length
train_data = pad_sequences(train_seq, maxlen=max_length)
val_data = pad_sequences(val_seq, maxlen=max_length)
test_data = pad_sequences(test_seq, maxlen=max_length)

In [ ]:
train_data[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 2047, 1877, 1676,   47, 5069,   19,   47,
       1285, 4958, 1470, 3057,  754,  104,  186, 3929, 6556, 1709,   27,
       7357, 1844,  142, 1309,  189,    1,    1,   58,   56,  228,   84,
         59,    9, 8094,  123,  638,   91,  213,  311,   22,  533,   16,
        228,    1,    1,   48,   31,  290,  213,   27, 2767, 2767,   27,
       5653,  584,  760, 4237,  206,    6,   27,   21,  228, 5283, 1057,
        746], dtype=int32)

In [ ]:
length_train_data = len(train_data[2])

print(length_train_data)

100


In [ ]:
train_data = np.array(train_data)  # Convert train_seq to a numpy array
shape_train_data = train_data[1].shape

print(shape_train_data)

(100,)


In [ ]:
# Loading the GloVe word embeddings
glove_path = 'drive/My Drive/data/glove.6B.100d.txt'

In [ ]:
import numpy as np
embedding_dim = 100  # Dimensionality of the word embeddings
embedding_index = {} #empty dictionary that will store the word embeddings. Each word will be associated with its corresponding embedding coefficients.
with open(glove_path, 'r') as f:
    for line in f:
        values = line.split() #Inside the loop, the current line is split into individual values using the split method, which splits the line based on whitespace. The resulting values are stored in the values list.
        word = values[0] #The first value in values (index 0) is assigned to the variable word. This represents the word itself.
        coefficients = np.asarray(values[1:], dtype='float32') #The word (word) and its corresponding coefficients (coefficients) are stored as a key-value pair in the embedding_index dictionary.
        embedding_index[word] = coefficients #After the loop ends, the embedding_index dictionary will contain the word embeddings for all words in the file.

In [ ]:
embedding_index['poor']

array([ 4.3508e-01,  4.9239e-01,  5.8290e-01,  2.2096e-02, -6.7535e-01,
        2.3478e-01, -6.2145e-01,  9.3806e-03, -1.4963e-01,  1.9908e-01,
       -2.3215e-01, -4.5859e-01,  1.2054e-01, -5.1395e-01,  3.1708e-01,
       -4.1910e-01,  5.1405e-02, -8.5546e-01,  2.6761e-01,  9.2187e-01,
       -1.3435e-02, -3.7181e-02,  9.1578e-01,  2.5520e-01,  7.8937e-02,
       -3.6655e-01, -7.6595e-01, -2.3991e-01, -4.7802e-01,  3.3219e-01,
        4.6679e-01,  1.5636e-01,  1.6042e-01,  8.7424e-02, -3.5321e-01,
        4.7255e-01,  1.2056e-01,  8.3116e-01, -6.0611e-02,  5.3739e-01,
       -6.1494e-01, -2.1570e-01, -1.8524e-01, -2.0223e-01,  2.3018e-01,
       -1.6400e-01,  2.3591e-01,  4.0722e-01, -2.7513e-02, -9.7465e-01,
       -6.6564e-01, -9.2480e-04, -4.7052e-01,  9.4277e-01, -3.2955e-02,
       -1.8189e+00,  4.1543e-01, -1.4880e-01,  1.7811e+00, -4.5183e-01,
       -3.6581e-01, -4.7664e-01, -4.6612e-01, -3.4389e-01,  7.3382e-01,
       -9.4791e-02,  3.0911e-01, -3.7634e-01,  9.0093e-01, -7.51

In [ ]:
# Creating an embedding matrix
word_index = tokenizer.word_index
num_words = min(vocab_size, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i < vocab_size:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix[1]

array([ 0.19788   ,  0.25264999, -0.28308001, -0.11095   , -0.73352998,
       -0.4752    ,  0.13349999, -0.14369   , -0.48716   ,  0.051429  ,
        0.60088998,  0.48181999,  0.12914   ,  0.69537002,  0.38631999,
       -0.33127999, -0.14934   , -0.71853   ,  0.75676   ,  0.18268   ,
       -0.14747   ,  0.33359   , -0.11076   ,  0.20818   ,  0.061527  ,
       -0.02967   ,  0.48225999, -0.065332  , -0.56993997, -0.093896  ,
        0.71170998,  0.48354   , -0.29899001,  0.39467001, -0.49208999,
        1.08340001,  0.10595   , -0.1287    ,  0.42798001,  0.69216001,
        0.60070997, -0.60420001, -0.72736001,  0.41709   ,  0.41475999,
        0.12808999,  0.20798001, -0.015487  ,  0.32878   ,  0.41328001,
       -0.73539001,  0.6674    , -0.28018001,  0.1635    , -0.81071001,
        0.29052001, -0.21743999,  0.81527001, -0.067178  , -0.38745001,
       -0.032753  , -0.2202    , -0.49691999,  0.58273   ,  1.18649995,
       -0.252     , -0.63714999,  0.45583001,  0.037555  ,  0.93

In [ ]:
#Standar LSTM
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_data, y_train, batch_size=64, epochs=10, validation_data=(val_data, y_val))

Epoch 1/10
626/626 [==============================] - 133s 209ms/step - loss: 0.4502 - accuracy: 0.7864 - val_loss: 0.3792 - val_accuracy: 0.8342
Epoch 2/10
626/626 [==============================] - 130s 207ms/step - loss: 0.3598 - accuracy: 0.8408 - val_loss: 0.3355 - val_accuracy: 0.8529
Epoch 3/10
626/626 [==============================] - 127s 203ms/step - loss: 0.3258 - accuracy: 0.8585 - val_loss: 0.3349 - val_accuracy: 0.8593
Epoch 4/10
626/626 [==============================] - 133s 212ms/step - loss: 0.3007 - accuracy: 0.8731 - val_loss: 0.3106 - val_accuracy: 0.8691
Epoch 5/10
626/626 [==============================] - 131s 210ms/step - loss: 0.2746 - accuracy: 0.8845 - val_loss: 0.3178 - val_accuracy: 0.8697
Epoch 6/10
626/626 [==============================] - 128s 204ms/step - loss: 0.2508 - accuracy: 0.8952 - val_loss: 0.3135 - val_accuracy: 0.8769
Epoch 7/10
626/626 [==============================] - 130s 207ms/step - loss: 0.2251 - accuracy: 0.9080 - val_loss: 0.3185 -

In [ ]:
# Evaluating the model
y_pred = model.predict(test_data)
y_pred_binary = np.round(y_pred).flatten()

157/157 [==============================] - 8s 46ms/step


In [ ]:
accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.8718
Precision: 0.866796875
Recall: 0.8809051210797936
F1-score: 0.8737940539476275
